In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

# DataTables, Indexes, Pandas, and Seaborn

## Some useful (free) resources

Introductory:

* [Getting started with Python for research](https://github.com/TiesdeKok/LearnPythonforResearch), a gentle introduction to Python in data-intensive research.

* [A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/index.html), by Jake VanderPlas, another quick Python intro (with notebooks).

Also,

* [Data Camp](https://www.datacamp.com)

* [Software Carpentry](https://software-carpentry.org/lessons)

* [Beginner Python Programming Tutorials on YouTube (Socratica)](https://www.youtube.com/watch?v=bY6m6_IIN94&list=PLi01XoE8jYohWFPpC17Z-wWhPOSuh8Er-)

Core Pandas/Data Science books:

* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/), by Jake VanderPlas.

* [Python for Data Analysis, 2nd Edition](http://proquest.safaribooksonline.com/book/programming/python/9781491957653), by  Wes McKinney, creator of Pandas. [Companion Notebooks](https://github.com/wesm/pydata-book)

* [Effective Pandas](https://github.com/TomAugspurger/effective-pandas), a book by Tom Augspurger, core Pandas developer.


Complementary resources:

* [An introduction to "Data Science"](https://github.com/stefanv/ds_intro), a collection of Notebooks by BIDS' [Stéfan Van der Walt](https://bids.berkeley.edu/people/st%C3%A9fan-van-der-walt).

* [Effective Computation in Physics](http://proquest.safaribooksonline.com/book/physics/9781491901564), by Kathryn D. Huff; Anthony Scopatz. [Notebooks to accompany the book](https://github.com/physics-codes/seminar). Don't be fooled by the title, it's a great book on modern computational practices with very little that's physics-specific.

* [Towards Data Science](https://towardsdatascience.com/), a large collection of articles for data science topics


OK, let's load and configure some of our core libraries (as an aside, you can find a nice visual gallery of available matplotlib sytles [here](https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html)).

These notes are adapted from the [Principles and Techniques of Data Science](https://www.textbook.ds100.org/ch/03/pandas_intro.html) textbook.


## Part 1: Getting Started

We will work with the Baby Names dataset available from the [Social Security Data Page](https://www.ssa.gov/data/): https://www.ssa.gov/OACT/babynames/index.html. We will pose a question, break the question down into high-level steps, then translate each step into Python code using `pandas` DataFrames. We begin by importing `pandas`:

In [1]:
import pandas as pd

In [2]:
# Use the shell command `head` to preview the first 10 items in the CSV
!head babynames.csv

Name,Sex,Count,Year
Mary,F,9217,1884
Anna,F,3860,1884
Emma,F,2587,1884
Elizabeth,F,2549,1884
Minnie,F,2243,1884
Margaret,F,2142,1884
Ida,F,1882,1884
Clara,F,1852,1884
Bertha,F,1789,1884


Now we can read in the data using `pd.read_csv` ([docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)).

In [5]:
baby = pd.read_csv('babynames.csv')


1891894

Note that for the code above to work, the `babynames.csv` file must be located in the same directory as this notebook. We can check what files are in the current folder by running the `ls` command-line tool (you can also omit the `!`, since it's one of the most-used commands; for more information, see [Chapter 1: IPython and Shell Commands](https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html)):

In [ ]:
!ls

When we use `pandas` to read in data, we get a DataFrame. A DataFrame is a tabular data structure where each column is labeled (in this case 'Name', 'Sex', 'Count', 'Year') and each row is labeled (in this case 0, 1, 2, ..., 1891893).

The labels of a DataFrame are called the *indexes* of the DataFrame and make many data manipulations easier.

## Indexes, Slicing, and Sorting

Let's use `pandas` to answer the following question:

**What were the five most popular baby names in 2016?**

### Breaking the Problem Down

We can decompose this question into the following simpler table manipulations:

1. Slice out the rows for the year 2016.
2. Sort the rows in descending order by Count.

Now, we can express these steps in `pandas`.

### Slicing using `.loc`

To select subsets of a DataFrame, we use the `.loc` slicing syntax. The first argument is the label of the row and the second is the label of the column:

In [ ]:
baby

In [ ]:
baby.loc[0, 'Name'] # Row labeled 1, Column labeled 'Name'

To slice out multiple rows or columns, we can use `:`. Note that **`.loc` slicing is inclusive, unlike Python's slicing**.

In [ ]:
# Get rows 1 through 5, columns Name through Count inclusive
baby.loc[1:5, 'Name':'Count']

### Slight tangent: exploring slicing, indexing and manipulation of DataFrames and Series

We will often want a single column from a DataFrame:

In [ ]:
baby.loc[:, 'Year']

Note that when we select a single column, we get a `pandas` Series. A Series is like a one-dimensional NumPy array since we can perform arithmetic on all the elements at once.

In [ ]:
baby.loc[:, 'Year'] * 2

To select out specific columns, we can pass a list into the `.loc` slice:

In [ ]:
# This is a DataFrame again
baby.loc[:, ['Name', 'Year']]

Selecting columns is common, so there's a shorthand.

In [ ]:
# Shorthand for baby.loc[:, 'Name']
baby['Name']

In [ ]:
# Shorthand for baby.loc[:, ['Name', 'Count']]
baby[['Name', 'Count']]

OK, end of tangent.

Let's get back to the original task.

#### Slicing rows using a predicate

To slice out the rows with year 2016, we will first create a Series containing `True` for each row we want to keep and `False` for each row we want to drop. This is simple because math and boolean operators on Series are applied to each element in the Series.

In [ ]:
# Series of years
baby['Year']

In [ ]:
# Compare each year with 2016
baby['Year'] == 2016

Once we have this Series of `True` and `False`, we can pass it into `.loc`.

In [ ]:
# We are slicing rows, so the boolean Series goes in the first
# argument to .loc
baby_2016 = baby.loc[baby['Year'] == 2016, :]
baby_2016

### Sorting Rows

The next step is the sort the rows in descending order by 'Count'. We can use the `sort_values()` function.

In [ ]:
sorted_2016 = baby_2016.sort_values('Count', ascending=False)
sorted_2016

Finally, we will use `.iloc` to slice out the first five rows of the DataFrame. **`.iloc` works like `.loc` but takes in numerical indices instead of labels**. It **does not include the right endpoint** in its slices, like Python's list slicing.

In [ ]:
# Get the value in the zeroth row, zeroth column
sorted_2016.iloc[0, 0]

In [ ]:
# Get the first five rows
sorted_2016.iloc[0:5]

## In Conclusion

We now have the five most popular baby names in 2016 and learned to express the following operations in `pandas`:

| Operation | `pandas` |
| --------- | -------  |
| Read a CSV file | `pd.read_csv()` |
| Slicing using labels or indices | `.loc` and `.iloc` |
| Slicing rows using a predicate | Use a boolean-valued Series in `.loc` |
| Sorting rows | `.sort_values()` |

## Part 2: Grouping and Pivoting

In this section, we will answer the question:

**What were the most popular male and female names in each year?**

Here's the Baby Names dataset once again:

In [ ]:
baby = pd.read_csv('babynames.csv')
baby.head()
# the .head() method outputs the first five rows of the DataFrame

### Breaking the Problem Down

We should first notice that the question in the previous section has similarities to this one; the question in the previous section restricts names to babies born in 2016 whereas this question asks for names in all years.

We once again decompose this problem into simpler table manipulations.

1. Group the `baby` DataFrame by 'Year' and 'Sex'.
2. For each group, compute the most popular name.

Recognizing which operation is needed for each problem is sometimes tricky. Usually, a convoluted series of steps will signal to you that there might be a simpler way to express what you want. If we didn't immediately recognize that we needed to group, for example, we might write steps like the following:

1. Loop through each unique year.
2. For each year, loop through each unique sex.
3. For each unique year and sex, find the most common name.

There is almost always a better alternative to looping over a `pandas` DataFrame. **In particular, looping over unique values of a DataFrame should usually be replaced with a group.**

### Grouping

To group in `pandas`. we use the `.groupby()` method.

In [ ]:
baby.groupby('Year')

`.groupby()` returns a strange-looking `DataFrameGroupBy` object. We can call `.agg()` on this object with an aggregation function in order to get a familiar output:

In [ ]:
# The aggregation function takes in a series of values for each group
# and outputs a single value
def length(series):
    return len(series)

# Count up number of values for each year. This is equivalent to
# counting the number of rows where each year appears.
baby.groupby('Year').agg(length)

You might notice that the `length` function simply calls the `len` function, so we can simplify the code above.

In [ ]:
baby.groupby('Year').agg(len)

The aggregation is applied to each column of the DataFrame, producing redundant information. We can restrict the output columns by slicing before grouping.

In [ ]:
year_rows = baby[['Year', 'Count']].groupby('Year').agg(len)
year_rows

# A further shorthand to accomplish the same result:
#
# year_counts = baby[['Year', 'Count']].groupby('Year').count()
#
# pandas has shorthands for common aggregation functions, including
# count, sum, and mean.

Note that the index of the resulting DataFrame now contains the unique years, so we can slice subsets of years using `.loc` as before:

In [ ]:
# Every twentieth year starting at 1880
year_rows.loc[1880:2016:20, :]

### Grouping on Multiple Columns

We can group on multiple columns to get groups based on unique pairs of values. To do this, pass in a list of column labels into `.groupby()`.

In [ ]:
grouped_counts = baby.groupby(['Year', 'Sex']).max()
grouped_counts

The code above computes the total number of babies born for each year and sex. Let's now use grouping by muliple columns to compute the most popular names for each year and sex. Since the data are already sorted in descending order of Count for each year and sex, we can define an aggregation function that returns the first value in each series. (If the data weren't sorted, we can call `sort_values()` first.)

In [ ]:
# The most popular name is simply the first one that appears in the series
def most_popular(series):
    return series.iloc[0]

baby_pop = baby.groupby(['Year', 'Sex']).agg(most_popular)
baby_pop

Notice that grouping by multiple columns results in multiple labels for each row. This is called a "multilevel index" and is tricky to work with. The important thing to know is that **`.loc` takes in a tuple for the row index instead of a single value**:

In [ ]:
baby_pop.loc[(2000, 'M'), 'Name']

But `.iloc` behaves the same as usual since it uses indices instead of labels:

In [ ]:
baby_pop.iloc[10:15, :]

### Pivoting

**If you group by two columns, you can often use pivot to present your data in a more convenient format.** Using a pivot lets you use one set of grouped labels as the columns of the resulting table.

To pivot, use the `pd.pivot_table()` function.

In [ ]:
pd.pivot_table(baby,
               index='Year',         # Index for rows
               columns='Sex',        # Columns
               values='Name',        # Values in table
               aggfunc=most_popular) # Aggregation function

Compare this result to the `baby_pop` table that we computed using `.groupby()`. We can see that the `Sex` index in `baby_pop` became the columns of the pivot table.

In [ ]:
baby_pop

## In Conclusion

We now have the most popular baby names for each sex and year in our dataset and learned to express the following operations in `pandas`:

| Operation | `pandas` |
| --------- | -------  |
| Group | `df.groupby(label)` |
| Group by multiple columns | `df.groupby([label1, label2])` |
| Group and aggregate | `df.groupby(label).agg(func)` |
| Pivot | `pd.pivot_table()` |